## 1. 라이브러리 불러오기

In [2]:
import sys
import glob
import cv2
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets, models
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

from torch.autograd import Variable

from matplotlib import pyplot as plt
from time import time

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold

# import wandb

# from augraphy import *

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

## 2. Custom Dataset

In [4]:
class ImageDataset(Dataset):
    def __init__(self, csv, path, album_transform=None, augraphy_transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path 
        self.album_transform = album_transform
        self.augraphy_transform = augraphy_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))                  

        
        if self.augraphy_transform:
            img = self.augraphy_transform(img)

        if self.album_transform:
            img = self.album_transform(image=img)['image']
        
        return img, target

## 3. Training Pipeline

In [5]:
def training(model, dataloader, dataset, device, criterion, optimizer, epoch, num_epochs):
    model.train()
    train_loss = 0.0
    preds_list = []
    targets_list = []

    tbar = tqdm(dataloader)
    for images, labels in tbar:
        images = images.type(torch.cuda.FloatTensor)
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(labels.detach().cpu().numpy())

        tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {loss.item():.4f}")

    train_loss = train_loss / (len(dataloader))
    train_acc = accuracy_score(preds_list, targets_list)
    train_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'train_loss' : train_loss,
        'train_acc' : train_acc,
        'train_f1' : train_f1
    }

    return model, metrics

def evaluation(model, dataloader, dataset, device, criterion, epoch, num_epochs):
    model.eval()
    valid_loss = 0.0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        tbar = tqdm(dataloader)
        for images, labels in tbar:
            images = images.type(torch.cuda.FloatTensor)
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            preds_list.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(labels.detach().cpu().numpy())

            tbar.set_description(f"Epcoh [{epoch+1}/{num_epochs}] Valid Loss : {valid_loss:.4f}")

    valid_loss /= len(dataloader)
    valid_acc = accuracy_score(preds_list, targets_list)
    valid_f1 = f1_score(preds_list, targets_list, average='macro')

    metrics = {
        'valid_loss' : valid_loss,
        'valid_acc' : valid_acc,
        'valid_f1' : valid_f1
    }

    return model, metrics

def training_loop(model, train_dataloader, valid_dataloader, train_dataset, valid_dataset, criterion, optimizer, device, num_epochs, model_path, model_name, patience):

    best_valid_loss = float('inf')
    valid_max_accuracy = -1
    valid_max_f1 = -1
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model, train_metrics = training(model, train_dataloader, train_dataset, device, criterion, optimizer, epoch, num_epochs)
        model, valid_metrics = evaluation(model, valid_dataloader, valid_dataset, device, criterion, epoch, num_epochs)

        monitoring_value = {
            'train_loss' : train_metrics['train_loss'],
            'train_accuracy' : train_metrics['train_acc'],
            'train_f1' : train_metrics['train_f1'],
            'valid_loss' : valid_metrics['valid_loss'],
            'valid_accuracy' : valid_metrics['valid_acc'],
            'valid_f1' : valid_metrics['valid_f1']
        }

        if valid_max_accuracy < valid_metrics['valid_acc']:
            valid_max_accuracy = valid_metrics['valid_acc']
        
        if valid_max_f1 < valid_metrics['valid_f1']:
            valid_max_f1 = valid_metrics['valid_f1']
            torch.save(model.state_dict(), model_path+f"/model_{model_name}.pt")

        if best_valid_loss > valid_metrics['valid_loss']:
            best_valid_loss = valid_metrics['valid_loss']
            early_stop_counter = 0

        else:
            early_stop_counter += 1
            
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss : {train_metrics['train_loss']:.4f}, Train Acc : {train_metrics['train_acc']:.4f}, 'Train F1 : {train_metrics['train_f1']:.4f}, Valid Loss : {valid_metrics['valid_loss']:.4f}, Valid Acc : {valid_metrics['valid_acc']:.4f}, Valid F1 : {valid_metrics['valid_f1']}")

        if early_stop_counter >= patience:
            print('Early Stopping!')        
            break

    return model, valid_max_accuracy, valid_max_f1


## 4. Data Load

In [6]:
img_csv_path = '/data/ephemeral/home/data/new_augdata.csv'
df_img = pd.read_csv(img_csv_path)


In [7]:
df_img

,ID,target2
0,augmented_0_a1ab865095b2d312_ljh.jpg,2
1,augmented_1_a1ab865095b2d312_ljh.jpg,2
2,augmented_2_a1ab865095b2d312_ljh.jpg,2
3,augmented_3_a1ab865095b2d312_ljh.jpg,2
4,augmented_4_a1ab865095b2d312_ljh.jpg,2
...,...,...
23545,augmented_3_dfa798b40397760b.jpg,11
23546,augmented_4_dfa798b40397760b.jpg,11
23547,augmented_5_dfa798b40397760b.jpg,11
23548,augmented_6_dfa798b40397760b.jpg,11


In [8]:
good_classes = set([0,1,2,5,6,8,15,16])
not_good_classes = set([3,4,7,9,10,11,12,13,14])
submission_path = '/data/ephemeral/home/data/sample_submission.csv'

In [9]:
good_classes_dict = {0:0, 1:1, 2:2, 5:3, 6:4, 8:5, 15:6, 16:7}
good_classes_dict_reverse = {0:0, 1:1, 2:2, 3:5, 4:6, 5:8, 6:15, 7:16}

In [10]:
bad_classes_dict = {3:0, 4:1, 7:2, 9:3, 10:4, 11:5, 12:6, 13:7, 14:8}
bad_classes_dict_reverse = {0:3, 1:4, 2:7, 3:9, 4:10, 5:11, 6:12, 7:13, 8:14}

In [11]:
# df_img_first = df_img.copy()
# for i in range(len(df_img_first)):
#     value = df_img_first.loc[i, 'target2']
#     if value in good_classes_dict:
#         df_img_first.loc[i, 'target2'] = good_classes_dict[value]
#     else:
#         df_img_first.loc[i, 'target2'] = 8


In [12]:
# df_img_first.to_csv("/data/ephemeral/home/data/first_classification_data.csv", index=False)

In [13]:
# df_not_good_classes = df_img[df_img['target2'].isin(not_good_classes)]
# df_not_good_classes = df_not_good_classes.reset_index(drop='index')
# for i in range(len(df_not_good_classes)):
#     value = df_not_good_classes.loc[i, 'target2']
#     df_not_good_classes.loc[i, 'target2'] = bad_classes_dict[value] 


In [14]:
# df_not_good_classes.to_csv("/data/ephemeral/home/data/second_classification_data.csv", index=False)


In [15]:
img_csv_path = '/data/ephemeral/home/data/first_classification_data.csv'
img_path = '/data/ephemeral/home/data/aug_dataset/aug_2'
test_img_path = '/data/ephemeral/home/data/test'
totensor_transform = A.Compose([A.Resize(380, 380), ToTensorV2()])
test_transform = A.Compose([
    A.Resize(380, 380),
    ToTensorV2()
])

train_dataset = ImageDataset(img_csv_path, img_path, album_transform=totensor_transform, augraphy_transform=None)
test_dataset = ImageDataset(submission_path, test_img_path, album_transform=test_transform, augraphy_transform=None)

print(len(train_dataset), len(test_dataset))

23550 3140


In [16]:
train_num, valid_num = int(len(train_dataset) * 0.8), int(len(train_dataset) * 0.2)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num])

print(len(train_dataset), len(valid_dataset))

18840 4710


In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 5. Train Model

In [18]:
model = timm.create_model('efficientnet_b4', pretrained=True)
in_features = model.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 9),
)

model.classifier = classifier

### Hyper Parameter 정의

In [19]:
class Cfg():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100
    batch_size=32
    model_path = '/data/ephemeral/home/models'

In [19]:
device = Cfg.device
model = Cfg.model
criterion = Cfg.criterion
optimizer = Cfg.optimizer 
num_epochs = Cfg.num_epochs
model_name = 'effb4-add_fc-first-classification'
model_path = Cfg.model_path

model, valid_max_accuracy, valid_max_f1 = training_loop(model, 
                                                        train_dataloader, 
                                                        valid_dataloader, 
                                                        train_dataset, 
                                                        valid_dataset, 
                                                        criterion, 
                                                        optimizer, 
                                                        device, 
                                                        num_epochs, 
                                                        model_path, 
                                                        model_name, 
                                                        20)

Epcoh [1/100] Valid Loss : 5.2611: 100%|██████████| 148/148 [00:32<00:00,  4.60it/s]


Epoch [1/100], Train Loss : 0.1956, Train Acc : 0.9420, 'Train F1 : 0.9180, Valid Loss : 0.0355, Valid Acc : 0.9885, Valid F1 : 0.9847023300293439


Epcoh [2/100] Valid Loss : 4.4951: 100%|██████████| 148/148 [00:32<00:00,  4.50it/s]


Epoch [2/100], Train Loss : 0.0530, Train Acc : 0.9848, 'Train F1 : 0.9796, Valid Loss : 0.0304, Valid Acc : 0.9902, Valid F1 : 0.985862495894014


Epcoh [3/100] Valid Loss : 6.0949: 100%|██████████| 148/148 [00:31<00:00,  4.73it/s]


...

Epoch [44/100], Train Loss : 0.0057, Train Acc : 0.9984, 'Train F1 : 0.9975, Valid Loss : 0.0252, Valid Acc : 0.9953, Valid F1 : 0.9937101767098826


Epcoh [45/100] Valid Loss : 1.5010: 100%|██████████| 148/148 [00:30<00:00,  4.84it/s]


Epoch [45/100], Train Loss : 0.0088, Train Acc : 0.9978, 'Train F1 : 0.9971, Valid Loss : 0.0101, Valid Acc : 0.9964, Valid F1 : 0.9954236616757784


Epcoh [46/100] Valid Loss : 1.4841: 100%|██████████| 148/148 [00:30<00:00,  4.80it/s]

Epoch [46/100], Train Loss : 0.0025, Train Acc : 0.9993, 'Train F1 : 0.9990, Valid Loss : 0.0100, Valid Acc : 0.9972, Valid F1 : 0.9970841250407178
Early Stopping!


In [20]:
effb4 = timm.create_model('efficientnet_b4', pretrained=True)
in_features = effb4.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 9),
)

effb4.classifier = classifier
effb4.load_state_dict(torch.load('/data/ephemeral/home/models/model_effb4-add_fc-first-classification.pt'))
effb4 = effb4.to(device)
effb4.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [21]:
preds_list = []

for images, labels in tqdm(test_dataloader):
    images = images.type(torch.cuda.FloatTensor)
    images = images.to(device)

    with torch.no_grad():
        preds = effb4(images)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:29<00:00,  3.31it/s]


In [22]:
pred_df = pd.read_csv("/data/ephemeral/home/data/sample_submission.csv")
pred_df['target'] = preds_list

In [23]:
pred_df.to_csv('/data/ephemeral/home/data/first_classificaion_pred.csv', index=False)

In [20]:
first_pred_df = pd.read_csv("/data/ephemeral/home/data/first_classificaion_pred.csv")
second_pred_df = first_pred_df[first_pred_df['target'] == 8].copy()
second_pred_df.to_csv("/data/ephemeral/home/data/second_classification_test.csv", index=False)
len(second_pred_df)

1644

In [21]:
second_pred_df

,ID,target
3,00471f8038d9c4b6.jpg,8
9,00c0dabb63ca7a16.jpg,8
11,00e15da96484eb94.jpg,8
13,0111a6728e9f8a73.jpg,8
15,01385f22f2490868.jpg,8
...,...,...
3131,ff611b061f5ceb6d.jpg,8
3132,ff6a9e516d685849.jpg,8
3133,ff70a399a80c1c96.jpg,8
3136,ffb54299b1ad4159.jpg,8


In [22]:
for i in range(len(first_pred_df)):
    value = first_pred_df.loc[i, 'target']
    if value in good_classes_dict_reverse:
        first_pred_df.loc[i, 'target'] = good_classes_dict_reverse[value]

In [23]:
img_csv_path = '/data/ephemeral/home/data/second_classification_data.csv'
img_path = '/data/ephemeral/home/data/aug_dataset/aug_2'

test_csv_path = "/data/ephemeral/home/data/second_classification_test.csv"
test_img_path = '/data/ephemeral/home/data/test'

totensor_transform = A.Compose([A.Resize(380, 380), ToTensorV2()])
test_transform = A.Compose([
    A.Resize(380, 380),
    ToTensorV2()
])

train_dataset = ImageDataset(img_csv_path, img_path, album_transform=totensor_transform, augraphy_transform=None)
test_dataset = ImageDataset(test_csv_path, test_img_path, album_transform=test_transform, augraphy_transform=None)

print(len(train_dataset), len(test_dataset))

12360 1644


In [24]:
train_num, valid_num = int(len(train_dataset) * 0.8), int(len(train_dataset) * 0.2)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num])

print(len(train_dataset), len(valid_dataset))

9888 2472


In [25]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [26]:
model = timm.create_model('efficientnet_b4', pretrained=True)
in_features = model.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 9),
)

model.classifier = classifier
model.load_state_dict(torch.load('/data/ephemeral/home/models/model_effb4-add_fc-first-classification.pt'))


<All keys matched successfully>

In [27]:
device = Cfg.device
model = Cfg.model
criterion = Cfg.criterion
optimizer = Cfg.optimizer 
num_epochs = Cfg.num_epochs
model_name = 'effb4-add_fc-second-classification'
model_path = Cfg.model_path

model, valid_max_accuracy, valid_max_f1 = training_loop(model, 
                                                        train_dataloader, 
                                                        valid_dataloader, 
                                                        train_dataset, 
                                                        valid_dataset, 
                                                        criterion, 
                                                        optimizer, 
                                                        device, 
                                                        num_epochs, 
                                                        model_path, 
                                                        model_name, 
                                                        20)

Epcoh [1/100] Valid Loss : 27.7756: 100%|██████████| 78/78 [00:16<00:00,  4.67it/s]


Epoch [1/100], Train Loss : 0.7784, Train Acc : 0.7120, 'Train F1 : 0.6804, Valid Loss : 0.3561, Valid Acc : 0.8592, Valid F1 : 0.8470801024911156


Epcoh [2/100] Valid Loss : 23.5150: 100%|██████████| 78/78 [00:15<00:00,  4.93it/s]


Epoch [2/100], Train Loss : 0.3249, Train Acc : 0.8840, 'Train F1 : 0.8768, Valid Loss : 0.3015, Valid Acc : 0.8904, Valid F1 : 0.8834932632754645


Epcoh [3/100] Valid Loss : 18.1091: 100%|██████████| 78/78 [00:15<00:00,  4.93it/s]


...

Epoch [47/100], Train Loss : 0.0118, Train Acc : 0.9966, 'Train F1 : 0.9963, Valid Loss : 0.1147, Valid Acc : 0.9709, Valid F1 : 0.9700243637188706


Epcoh [48/100] Valid Loss : 14.4784: 100%|██████████| 78/78 [00:16<00:00,  4.68it/s]


Epoch [48/100], Train Loss : 0.0170, Train Acc : 0.9961, 'Train F1 : 0.9962, Valid Loss : 0.1856, Valid Acc : 0.9600, Valid F1 : 0.9585689261253513


Epcoh [49/100] Valid Loss : 11.1450: 100%|██████████| 78/78 [00:16<00:00,  4.75it/s]

Epoch [49/100], Train Loss : 0.0330, Train Acc : 0.9911, 'Train F1 : 0.9909, Valid Loss : 0.1429, Valid Acc : 0.9640, Valid F1 : 0.9635702762239546
Early Stopping!


In [28]:
effb4 = timm.create_model('efficientnet_b4', pretrained=True)
in_features = effb4.classifier.in_features
classifier = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 9),
)

effb4.classifier = classifier
effb4.load_state_dict(torch.load('/data/ephemeral/home/models/model_effb4-add_fc-second-classification.pt'))
effb4 = effb4.to(device)
effb4.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [29]:
preds_list = []

for images, labels in tqdm(test_dataloader):
    images = images.type(torch.cuda.FloatTensor)
    images = images.to(device)

    with torch.no_grad():
        preds = effb4(images)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 52/52 [00:16<00:00,  3.24it/s]


In [30]:
second_pred_df['target'] = preds_list

In [31]:
second_pred_df

,ID,target
3,00471f8038d9c4b6.jpg,6
9,00c0dabb63ca7a16.jpg,5
11,00e15da96484eb94.jpg,2
13,0111a6728e9f8a73.jpg,3
15,01385f22f2490868.jpg,1
...,...,...
3131,ff611b061f5ceb6d.jpg,5
3132,ff6a9e516d685849.jpg,0
3133,ff70a399a80c1c96.jpg,5
3136,ffb54299b1ad4159.jpg,4


In [32]:
len(second_pred_df)

1644

In [33]:
for i in second_pred_df.index:
    value = second_pred_df.loc[i, 'target']
    second_pred_df.loc[i, 'target'] = bad_classes_dict_reverse[value]



In [34]:
second_pred_df

,ID,target
3,00471f8038d9c4b6.jpg,12
9,00c0dabb63ca7a16.jpg,11
11,00e15da96484eb94.jpg,7
13,0111a6728e9f8a73.jpg,9
15,01385f22f2490868.jpg,4
...,...,...
3131,ff611b061f5ceb6d.jpg,11
3132,ff6a9e516d685849.jpg,3
3133,ff70a399a80c1c96.jpg,11
3136,ffb54299b1ad4159.jpg,10


In [35]:
for idx in second_pred_df.index:
    first_pred_df.loc[idx, 'target'] = second_pred_df.loc[idx, 'target']

In [36]:
first_pred_df['target'].value_counts()

target
10    224
7     216
11    204
6     204
16    201
5     200
15    200
0     200
8     200
9     200
4     199
2     199
12    184
13    154
3     154
14    109
1      92
Name: count, dtype: int64

In [37]:
first_pred_df.to_csv('/data/ephemeral/home/submission/effb4_0213.csv', index=False)

In [38]:
first_pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,6
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
